# Data-X Spring 2019: Homework 7

### Webscraping



In this homework, you will do some exercises with web-scraping.

## Name: Keshav Kothari

## SID: 3034344473



### Fun with Webscraping & Text manipulation


## 1. Statistics in Presidential Debates

Your first task is to scrape Presidential Debates from the Commission of Presidential Debates website: https://www.debates.org/voter-education/debate-transcripts/

To do this, you are not allowed to manually look up the URLs that you need, instead you have to scrape them. The root url to be scraped is the one listed above, namely: https://www.debates.org/voter-education/debate-transcripts/


1. By using `requests` and `BeautifulSoup` find all the links / URLs on the website that links to transcriptions of **First Presidential Debates** from the years [1988, 1984, 1976, 1960]. In total you should find 4 links / URLs that fulfill this criteria. **Print the urls.**

2. When you have a list of the URLs your task is to create a Data Frame with some statistics (see example of output below):
    1. Scrape the title of each link and use that as the column name in your Data Frame. 
    2. Count how long the transcript of the debate is (as in the number of characters in transcription string). Feel free to include `\` characters in your count, but remove any breakline characters, i.e. `\n`. You will get credit if your count is +/- 10% from our result.
    3. Count how many times the word **war** was used in the different debates. Note that you have to convert the text in a smart way (to not count the word **warranty** for example, but counting **war.**, **war!**, **war,** or **War** etc.
    4. Also scrape the most common used word in the debate, and write how many times it was used. Note that you have to use the same strategy as in C in order to do this.
    
    **Print your final output result.**
    
**Tips:**

___

In order to solve the questions above, it can be useful to work with Regular Expressions and explore methods on strings like `.strip(), .replace(), .find(), .count(), .lower()` etc. Both are very powerful tools to do string processing in Python. To count common words for example I used a `Counter` object and a Regular expression pattern for only words, see example:

```python
    from collections import Counter
    import re

    counts = Counter(re.findall(r"[\w']+", text.lower()))
```

Read more about Regular Expressions here: https://docs.python.org/3/howto/regex.html
    
    
**Example output of all of the answers to Question 1.2:**


![pres_stats_2](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/presidents_stats_2.jpg)



----

.




In [1]:
# your code here
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>")) 
from __future__ import division, print_function
import requests
import bs4 as bs

In [2]:
source = requests.get("https://www.debates.org/voter-education/debate-transcripts/") 
soup = bs.BeautifulSoup(source.content, features='html.parser')
links  = soup.find_all('a')

In [3]:
debate_links = []
print ('Relevant Links:')
for url in links:
    if 'First' in url.text and 'Presidential' in url.text and 'Debate' in url.text and ('1960' in url.text or '1976' in url.text or '1984' in url.text or '1988' in url.text):
        print ("https://www.debates.org" + url.get('href'))
        debate_links.append(url.get('href'))

Relevant Links:
https://www.debates.org/voter-education/debate-transcripts/september-25-1988-debate-transcript/
https://www.debates.org/voter-education/debate-transcripts/october-7-1984-debate-transcript/
https://www.debates.org/voter-education/debate-transcripts/september-23-1976-debate-transcript/
https://www.debates.org/voter-education/debate-transcripts/september-26-1960-debate-transcript/


In [4]:
debate_content = {}
titles = []
for dl in debate_links:
    source = requests.get("https://www.debates.org" + dl)
    soup = bs.BeautifulSoup(source.content, features='html.parser') 
    titles.append(soup.title.text)
    debate_content[soup.title.text] = soup

In [5]:
import pandas as pd
df = pd.DataFrame(columns=titles)

In [6]:
list_count = []
for t in titles:
    s=0
    for p in debate_content[t].find_all('p'):
        s=s + len(p.text) - p.text.count('\n')
    list_count.append(s)
df.loc["Debate Char Length"] =  list_count

In [7]:
import re
from collections import Counter

war_count = []
word_list = []
for t in titles:
    s=[]
    for p in debate_content[t].find_all('p'):
        s = s + (re.sub('\n','',re.sub(r'[^\w\s]','',p.text.lower()))).split(' ')
    war_count.append(s.count('war'))
    word_list.append(s)
df.loc["War Count"] =  war_count

In [8]:
word_ = []
num_count = []
for wl in word_list:
    word_.append(Counter(wl).most_common(1)[0][0])
    num_count.append(Counter(wl).most_common(1)[0][1])
df.loc['Most Common Word'] = word_
df.loc['Most Common Word Count'] = num_count

In [9]:
df

,"CPD: September 25, 1988 Debate Transcript","CPD: October 7, 1984 Debate Transcript","CPD: September 23, 1976 Debate Transcript","CPD: September 26, 1960 Debate Transcript"
Debate Char Length,87529,86549,80776,60978
War Count,7,2,7,3
Most Common Word,the,the,the,the
Most Common Word Count,800,868,857,780


    
## 2. Download and read in specific line from many data sets

Scrape the first 27 data sets from this URL http://people.sc.fsu.edu/~jburkardt/datasets/regression/ (i.e.`x01.txt` - `x27.txt`). Then, save the 5th line in each data set, this should be the name of the data set author (get rid of the `#` symbol, the white spaces and the comma at the end). 

Count how many times (with a Python function) each author is the reference for one of the 27 data sets. Showcase your results, sorted, with the most common author name first and how many times he appeared in data sets. Use a Pandas DataFrame to show your results, see example. **Print your final output result.**

**Example output of the answer for Question 2:**

![author_stats](https://github.com/ikhlaqsidhu/data-x/raw/master/x-archive/misc/hw2_imgs_spring2018/data_authors.png)


In [10]:
name_list=[]
for i in range(1,28):
    source = requests.get("http://people.sc.fsu.edu/~jburkardt/datasets/regression/x"+str(i).zfill(2)+".txt") 
    soup = bs.BeautifulSoup(source.content, features='html.parser')
    s = str(soup).split('\n#    ')
    name_list.append(s[1].replace('and',','))
name_list

['Helmut Spaeth,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'R J Freund , P D Minton,',
 'D G Kleinbaum , L L Kupper,',
 'Helmut Spaeth,',
 'D G Kleinbaum , L L Kupper,',
 'K A Brownlee,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'S Chatterjee , B Price,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'R J Freund , P D Minton,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'Helmut Spaeth,',
 'S Chatterjee, B Price,',
 'S Chatterjee, B Price,',
 'S Chatterjee, B Price,',
 'S C Narula, J F Wellington,',
 'S C Narula, J F Wellington,']

In [11]:
name_count={}
for names in name_list:
    name = names.split(',')
    for n in name:
        m = n.strip()
        if len(m)>0:
            if m not in name_count:
                name_count[m]=0
            name_count[m]+=1
name_count

{'Helmut Spaeth': 16,
 'R J Freund': 2,
 'P D Minton': 2,
 'D G Kleinbaum': 2,
 'L L Kupper': 2,
 'K A Brownlee': 1,
 'S Chatterjee': 4,
 'B Price': 4,
 'S C Narula': 2,
 'J F Wellington': 2}

In [12]:
df = pd.DataFrame.from_dict(name_count,orient='index',columns=['Count'])
df.index.names = ['Authors']
df=df.sort_values(by=['Count'],ascending=False)
df

,Count
Authors,
Helmut Spaeth,16
S Chatterjee,4
B Price,4
R J Freund,2
P D Minton,2
D G Kleinbaum,2
L L Kupper,2
S C Narula,2
J F Wellington,2
